In [ ]:
import pandas as pd

In [ ]:
dataset_path = r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset"

In [ ]:
# Import datasets
customers = pd.read_csv(dataset_path + '/customers.csv')
order_items = pd.read_csv(dataset_path +'/order_items.csv')
order_payments = pd.read_csv(dataset_path +'/order_payments.csv')
order_reviews = pd.read_csv(dataset_path +'/order_reviews.csv')
orders = pd.read_csv(dataset_path +'/orders.csv')
products = pd.read_csv(dataset_path +'/products.csv')
sellers = pd.read_csv(dataset_path +'/sellers.csv')

data = {'customers': customers,
        'order_items': order_items, 
        'order_payments': order_payments,
        'order_reviews': order_reviews,
        'orders': orders,
        'products': products,
        'sellers': sellers}

1) Total Price: Sum of product price and freight value.

In [ ]:
# We will create a new column called total price. 
order_items['total_price'] = order_items['price'] + order_items['freight_value']

In [ ]:
order_items[['total_price', 'price', 'freight_value']].head()

2) Delivery Time: Difference between delivery date and purchase date

In [ ]:
# First we want to set the purchase and delivery to datetime.  
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

In [ ]:
# Next we want to merge the order_items table with order table to get the purchase and delivery date.
order_items = order_items.merge(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']], on='order_id', how='left')

In [ ]:
# Calculate delivery time in days
order_items['delivery_time'] = (order_items['order_delivered_customer_date'] -order_items['order_purchase_timestamp']).dt.days


In [ ]:
order_items[['order_id', 'delivery_time']].head()

3) Payment Count: Sum of payment installments for each order.

In [ ]:
# We will group by order_id and sum the payment installments
payment_sum = order_payments.groupby('order_id')['payment_installments'].sum().reset_index()
payment_sum.columns = ['order_id', 'total_payments_count']


In [ ]:
# Merge the payment data into the order_items table
order_items = order_items.merge(payment_sum, on='order_id', how='left')
order_items[['order_id', 'total_payments_count']].head()


 4. Profit Margin: Subtract freight value from product price

In [ ]:
# We will substract freight value from the product price
order_items['profit_margin'] = order_items['price'] - order_items['freight_value']

In [ ]:
# Get all the new columns
order_items[['order_id', 'total_price', 'delivery_time', 'total_payments_count', 'profit_margin']].head()

In [ ]:
order_items.to_csv(r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset\order_items.csv", index=False)